In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import importlib
import transformer_type
from transformer_type.dataset import get_data
import torch
import pandas as pd

if torch.cuda.is_available(): device = torch.device ("cuda")
elif torch.backends.mps.is_available(): device = torch.device("mps")
else: device = torch.device('cpu')

In [2]:
# params = [120, 1, 64, 300, 32, 512, 2, 1e-5, 0.1]
params = {'input_window' : [120],
         'pred_step' : [30],
         'batch_size' : [32],
         'epochs' : [100],
         'feature_size' : [32],
         'd_ff' : [256],
         'num_layers' : [3],
         'lr' : [1e-4],
         'device' : [device],
         'dropout' : [0.1],
         'is_save' : [True]}

In [3]:
DATA_PATH = "./data/kaggle_data/daily_data/"

In [4]:

# make dataset

def data_extract(df_dict, df, name, cols):
       for col in cols:
              df_dict[f"{name}_{col}"] = df.loc[df[col] != 0, col]
       return df_dict

train_data_list = []
val_data_list = []

pmj_df = pd.read_csv(DATA_PATH + "pjm_elec_daily.csv")
pmj_cols = ['AEP', 'COMED', 'DAYTON', 'DEOK', 'DOM', 'DUQ', 'FE', 'NI', 'PJME', 'PJMW'] # 'PJM_Load', 'EKPC'

df_dict = {}

df_dict = data_extract(df_dict, pmj_df, 'PMJ', pmj_cols)

for df_name in df_dict:
    df = df_dict[df_name]
    train_data, val_data, scaler = get_data(df, params['input_window'][0], 1)
    train_data = train_data.to(device)
    val_data = val_data.to(device)
    print(df_name, len(train_data), len(val_data))
    train_data_list.append((train_data, df_name, scaler))
    val_data_list.append((val_data, df_name, scaler))

PMJ_AEP 3923 890
PMJ_COMED 2096 434
PMJ_DAYTON 3923 890
PMJ_DEOK 1804 361
PMJ_DOM 3753 848
PMJ_DUQ 3849 872
PMJ_FE 1975 404
PMJ_NI 1828 367
PMJ_PJME 4726 1091
PMJ_PJMW 4654 1073


In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(series, order=(0,1,1))
model_fit = model.fit(trend='c',full_output=True, disp=1)
print(model_fit.summary())